In [ ]:
import win32com.client as win32
import pandas as pd
import xlwings as xw
import os
import PyPDF2 as pdf
from python_calamine.pandas import pandas_monkeypatch
from time import sleep

pandas_monkeypatch()

In [3]:
path_premissa = r'F:\Silvicultura\01. SP\PROCESSOS\COI Silvicultura - Edicao\01. Programações & Controles\016. Insumos\Novo Fluxo Insumos\E-mails EPS.xlsx'

In [4]:
path_report = r'F:\Silvicultura\01. SP\PROCESSOS\COI Silvicultura - Edicao\01. Programações & Controles\016. Insumos\Novo Fluxo Insumos\Análise Insumos.xlsx'

aba_report = 'Controle NFs'

aba_notas = 'NotasFiscais'

In [5]:
path_anexo2 = r'F:\Silvicultura\01. SP\PROCESSOS\COI Silvicultura - Edicao\01. Programações & Controles\016. Insumos\Novo Fluxo Insumos\Report Apontamentos.xlsx'

In [6]:
premissa = pd.read_excel(path_premissa,sheet_name='email')

c:\Users\gamoura\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [7]:
#Contatos Próprio 1

out_p1 = premissa[['EPS','Contatos Próprio 1']]

out_p1 = out_p1[pd.notna(out_p1['Contatos Próprio 1'])]

out_p1 = out_p1.rename({'Contatos Próprio 1':'Contatos EPS'},axis=1)

In [8]:
#Contatos Próprio 2
out_p2 = premissa[['EPS','Contatos Próprio 2']]

out_p2 = out_p2[pd.notna(out_p2['Contatos Próprio 2'])]

out_p2 = out_p2.rename({'Contatos Próprio 2':'Contatos EPS'},axis=1)

In [9]:
#Emails em cópia

copia = premissa[pd.notna(premissa['Cópia'])][['Cópia']].copy()


In [10]:
#Corpo do email

corpo_email = pd.read_excel( path_premissa,sheet_name='Corpo email')


#Assunto
assunto = corpo_email[['Assunto']].copy()

#Corpo email
corpo_email = corpo_email[['Corpo do Email']]


In [11]:
#Premissas

premissa = premissa[['EPS','Contatos EPS']]

contatos = pd.concat([out_p1,out_p2,premissa])

In [12]:
contatos = contatos.sort_values('EPS', ascending=True)

#Removendo itens duplicados
contatos = contatos.drop_duplicates(['EPS', 'Contatos EPS'],keep='first')


In [13]:
caminho_output = r"F:\Silvicultura\01. SP\PROCESSOS\COI Silvicultura - Edicao\01. Programações & Controles\016. Insumos\Novo Fluxo Insumos\export\Output.pdf"

In [14]:
#Criar função para exportar um excel para cada eps de acordo com as informações da dinâmica


notas_fiscais = pd.read_excel(path_report,
                               sheet_name= aba_notas,
                               engine= 'calamine', 
                               usecols=['Status NF', 'EPS', 'Semana Emissão','NF','Insumo','Quantidade com EPS','Saldo NF tratado','Saldo NF'],
                               dtype={ 'NF': 'object'})

notas_fiscais = notas_fiscais.rename({
    'Quantidade com EPS':'Quantidade Retirada',
    'Saldo NF tratado' : 'Saldo Notas',
    'Saldo NF': 'Saldo Real NFs'
}, axis=1)



agg_nf = {'Quantidade Retirada': 'sum' ,
           'Saldo Notas' : 'sum',
           'Saldo Real NFs' : 'sum'}

notas_fiscais = notas_fiscais.groupby(['EPS', 'Semana Emissão', 'NF', 'Insumo']).agg(agg_nf).reset_index()



#Mudando o tipo de dado e garantindo que tenha 9 dígitos

notas_fiscais['NF'] = notas_fiscais['NF'].astype('str')

notas_fiscais['NF'] = notas_fiscais['NF'].str.zfill(9)


ValueError: Unknown engine: calamine

In [ ]:
caminho_output_export = r"F:\Silvicultura\01. SP\PROCESSOS\COI Silvicultura - Edicao\01. Programações & Controles\016. Insumos\Novo Fluxo Insumos\export"

In [ ]:
#Exportar as notas ficais

def export_notas (notas_fiscais):
    for i in notas_fiscais['EPS']:
        notas_fiscais_filtro = notas_fiscais[notas_fiscais['EPS'] == i]
        notas_fiscais_filtro.to_excel( os.path.join(caminho_output_export, f'notas_fiscais_{i}.xlsx'),sheet_name=f'notas_{i}', index=False)


In [ ]:
export_notas(notas_fiscais=notas_fiscais)

In [ ]:
eps_lista = []

def excel_insumo(path, aba, nome_tabela, campo_filtro, valor_filtro, coluna_eps):
    with xw.App(visible=False) as app:
        
        # Abrir o Excel
        workbook = app.books.open(path, read_only=True)
        worksheet = workbook.sheets[aba]
       
        # Acessar a Tabela Dinâmica
        tabela_dinamica = worksheet.api.PivotTables(nome_tabela)

        # # Aplicar filtro para o campo (ex: "Status NF")
        # campo_filtro_obj = tabela_dinamica.PivotFields(campo_filtro)
        # campo_filtro_obj.ClearAllFilters()
        # campo_filtro_obj.CurrentPage = valor_filtro

        #Listar itens do campo

        campo_eps = tabela_dinamica.PivotFields(coluna_eps)
        


        
        
        itens = [item.Name for item in campo_eps.PivotItems()]

        eps_lista.extend(itens)


        for i in itens:
            # Limpar filtros anteriores
            campo_eps.ClearAllFilters()

            # Iterar sobre os itens da coluna_eps e aplicar o filtro para cada um
            for item in campo_eps.PivotItems():
                if item.Name == i:
                    item.Visible = True  # Exibir o item desejado
                else:
                    item.Visible = False  # Ocultar os itens que não são o desejado

        
            # Gerar caminho para salvar o PDF com o nome do item filtrado
            caminho_pdf = caminho_output.replace(".pdf", f"_{i}.pdf")
           

            #Mudar a orientação da página
                        
            worksheet.api.PageSetup.FitToPagesWide = 1
            
            worksheet.api.PageSetup.Orientation = 2

            worksheet.api.PageSetup.PaperSize = 8
                      

            worksheet.api.ExportAsFixedFormat(0, caminho_pdf)

       

In [18]:
excel_insumo(path=path_report, aba=aba_report, nome_tabela='SaldoNFS', campo_filtro='Status NF', valor_filtro='Pendente',coluna_eps='EPS')

In [19]:

lista_itens = r"F:\Silvicultura\01. SP\PROCESSOS\COI Silvicultura - Edicao\01. Programações & Controles\016. Insumos\Novo Fluxo Insumos\export"

In [20]:
#Editar PDF

def editar_pdf(caminho_pdf):
    
    for i in os.listdir(caminho_pdf):
        if i.lower().endswith('.pdf'):


            file = os.path.join(caminho_pdf, i)
        
            with open(file, 'rb') as pdf_file:
                reader = pdf.PdfReader(pdf_file)
                
                writer = pdf.PdfWriter()

                    # Iterar sobre as páginas e adicionar apenas as que não estão vazias
                for page_number, page in enumerate(reader.pages, start=1):
                    text = page.extract_text()

                    # Adicionar a página ao novo PDF se não estiver vazia
                    if text.strip():  # Verificar se a página contém texto
                        writer.add_page(page)

                with open(file, 'wb') as output_pdf:
                    writer.write(output_pdf)

editar_pdf(lista_itens)

In [21]:
eps_lista = ['TESTE']

In [22]:
def envio_email_pers(contatos, eps_lista,caminho_anexo):
      # Mover para fora do loop para armazenar todos os e-mails
    
    for eps in eps_lista:


        anexo = []

        for arquivo in os.listdir(caminho_anexo):
            if eps in arquivo and arquivo.lower().endswith('.pdf'):
                caminho_anexo_envio = os.path.join(caminho_anexo, arquivo)
                anexo.append(caminho_anexo_envio)
        print(anexo)


        anexo_xlsx = []

        for arquivo in os.listdir(caminho_anexo):
            if eps in arquivo and arquivo.lower().endswith('.xlsx'):
                caminho_anexo_envio = os.path.join(caminho_anexo, arquivo)
                anexo_xlsx.append(caminho_anexo_envio)
        print(anexo_xlsx)



        email_responsavel = []

        for k, v in contatos.iterrows():
          
            if eps == v['EPS']:               

                email_responsavel.append(v['Contatos EPS'])                

                #emails_flat = ["".join(email) for email in email_responsavel]                

                email_to = " ; ".join(email_responsavel)
                print(email_to)        

        print(eps)


        email = win32.Dispatch('Outlook.Application').CreateItem(0)
        
        email.To = email_to

        email.Subject = assunto['Assunto'].iloc[0] + " - " + str(eps)

        email.Body = corpo_email['Corpo do Email'].iloc[0] 

        email.CC = " ; ".join(copia['Cópia'])

        email.Attachments.Add(anexo[0])

        email.Attachments.Add(anexo_xlsx[0])

        email.Attachments.Add(path_anexo2)
        
        #Ativar quando camila estiver executando
        email.SentOnBehalfOfName = "insumo_silvi@bracell.com"
        




        email.Send()

        print(f'O e-mail foi enviado para a EPS {eps}')
    #return email_responsavel


In [23]:
envio_email_pers(contatos=contatos, eps_lista=eps_lista, caminho_anexo=lista_itens)

['F:\\Silvicultura\\01. SP\\PROCESSOS\\COI Silvicultura - Edicao\\01. Programações & Controles\\016. Insumos\\Novo Fluxo Insumos\\export\\Output_TESTE.pdf', 'F:\\Silvicultura\\01. SP\\PROCESSOS\\COI Silvicultura - Edicao\\01. Programações & Controles\\016. Insumos\\Novo Fluxo Insumos\\export\\TESTE.pdf']
['F:\\Silvicultura\\01. SP\\PROCESSOS\\COI Silvicultura - Edicao\\01. Programações & Controles\\016. Insumos\\Novo Fluxo Insumos\\export\\notas_fiscais_TESTE.xlsx']
gamoura@bracell.com
TESTE
O e-mail foi enviado para a EPS TESTE


In [24]:
sleep(30)

In [25]:
# #Apagar PDF 


# def apagar_pdf(caminho_output):
#     for i in os.listdir(caminho_output):
#         os.remove(os.path.join(lista_itens,i))
        

# #apagar_pdf(remove_itens)